# IE 582 Homework 04 - Dorukhan Kılınç 2017402093


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

## Task: Multiple Instance Learning

In this task, we are asked to suggest two bag-level representations fot the given multiple instance learning problem. In this problem, the data we are given consists of bags of instances. Class labels are assigned to each bag, i.e. all the instances which are in the same bag have the same label. A bag is labeled as 0 if there are no instances labeled as 1, and 1 otherwise. 

In addition, we are asked to build 2 models of our choice and validata/evaluate them based on their 10 fold cv accuracy scores. Therefore, penalized logistic regression and knn classifier will be used for evaluation.

### First Representation

Notice that we can surely label an instance as class 0 if it belongs to a bag whose label is 0. However, we can not surely say whether instances in a bag labeled as class 1 are of class 1 or 0. Thus, the first representation takes different approaches to represent these bags. If a bag is labeled as of class 0, we simply take the average of the features as suggested in the task definition. However, this strategy would not work in the class 1 bags since even the existance of one class 1 instance results in class 1 label. Therefore, to represent these bags euclidean distance between the bag instances is taken and the instance which is the furthest to the others is selected as the representative of the bag. By doing this, we are simply selecting the most outlier instance as the representative and therefore taking into consideration the cases at which there is only one class 1 instance.

In [3]:
data = pd.read_csv("Musk1.csv", header = None)

In [4]:
data

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
0,1,1,42,-198,-109,-75,-117,11,23,-88,...,-238,-74,-129,-120,-38,30,48,-37,6,30
1,1,1,42,-191,-142,-65,-117,55,49,-170,...,-238,-302,60,-120,-39,31,48,-37,5,30
2,1,1,42,-191,-142,-75,-117,11,49,-161,...,-238,-73,-127,-120,-38,30,48,-37,5,31
3,1,1,42,-198,-110,-65,-117,55,23,-95,...,-238,-302,60,-120,-39,30,48,-37,6,30
4,1,2,42,-198,-102,-75,-117,10,24,-87,...,-238,-73,-127,51,128,144,43,-30,14,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,0,92,49,-199,-161,29,-95,-86,-48,2,...,-220,-246,-209,33,152,134,47,-43,-15,-10
472,0,92,38,-123,-139,30,-117,-88,214,-13,...,-236,-226,-210,20,55,119,79,-28,4,74
473,0,92,43,-102,-20,-101,-116,200,-166,66,...,114,32,136,-15,143,121,55,-37,-19,-36
474,0,92,39,-58,27,31,-117,-92,85,21,...,-228,-232,-206,13,45,116,79,-28,3,74


In [5]:
columns = {0:"Class", 1:"Bag"}
for i in range(166):
    columns[i+2] = "Feature"+str(i+1)
    
data.rename(columns = columns, inplace = True)
data

,Class,Bag,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,...,Feature157,Feature158,Feature159,Feature160,Feature161,Feature162,Feature163,Feature164,Feature165,Feature166
0,1,1,42,-198,-109,-75,-117,11,23,-88,...,-238,-74,-129,-120,-38,30,48,-37,6,30
1,1,1,42,-191,-142,-65,-117,55,49,-170,...,-238,-302,60,-120,-39,31,48,-37,5,30
2,1,1,42,-191,-142,-75,-117,11,49,-161,...,-238,-73,-127,-120,-38,30,48,-37,5,31
3,1,1,42,-198,-110,-65,-117,55,23,-95,...,-238,-302,60,-120,-39,30,48,-37,6,30
4,1,2,42,-198,-102,-75,-117,10,24,-87,...,-238,-73,-127,51,128,144,43,-30,14,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,0,92,49,-199,-161,29,-95,-86,-48,2,...,-220,-246,-209,33,152,134,47,-43,-15,-10
472,0,92,38,-123,-139,30,-117,-88,214,-13,...,-236,-226,-210,20,55,119,79,-28,4,74
473,0,92,43,-102,-20,-101,-116,200,-166,66,...,114,32,136,-15,143,121,55,-37,-19,-36
474,0,92,39,-58,27,31,-117,-92,85,21,...,-228,-232,-206,13,45,116,79,-28,3,74


In [6]:
def representation1(bag):
    bag_array = np.array(bag)
    class_id = bag_array[0,0]
    bag_id = bag_array[0,1]
    features = np.array(bag.drop(columns = ["Class", "Bag"]))
    if class_id == 0:
        avg_features = np.average(features.T, axis = 1)
        return np.concatenate((np.array([class_id,bag_id]), avg_features))
    else: 
        dists = euclidean_distances(features)
        total_distances = dists.sum(axis = 1)
        return bag_array[np.argmax(total_distances),:]

In [7]:
rep1 = np.zeros((92,168))
for bag_id in range(1,93):
    bag = data[data["Bag"]==bag_id]
    rep1[bag_id - 1,:] = representation1(bag)

#### Penalized Logistic Regression Applied to the First Representation

In [9]:
X1 = rep1[:,2:]
y1 = rep1[:,0]
model1 = LogisticRegressionCV(cv=10, random_state=0,
                              max_iter = 1000).fit(X1, y1)


In [25]:
print("Penalization terms:")
print(model1.Cs_)
print()
print("Corresponding average cross-validation scores:")
print(np.average(model1.scores_[1].T,axis = 1))
print()
print("Best penalization term")
print(model1.C_)

Penalization terms:
[1.00000000e-04 7.74263683e-04 5.99484250e-03 4.64158883e-02
 3.59381366e-01 2.78255940e+00 2.15443469e+01 1.66810054e+02
 1.29154967e+03 1.00000000e+04]

Corresponding average cross-validation scores:
[0.80666667 0.84666667 0.83555556 0.83555556 0.82555556 0.81444444
 0.80444444 0.80444444 0.80444444 0.80444444]

Best penalization term
[0.00077426]


Here above is the cross-validation scores for penalized logistic regression and most suitable penalty term. 0.84666667 is the corresponding cv score. 

#### KNN Classifier Applied to the First Representation

For this model, there are two parameters to tune, number of neighbors to consider when classifying (k) and the parameter of the minkowski distance. For k values I have chosen odd number up to 9 and for p values I have chosen 1 and 2 (Manhattan and Euclidean distance).

In [28]:
#Cross validation function for parameter tuning
def cv(model, X, y):
    
    rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=0)
    scores = []
    
    
    for train_index, test_index in rskf.split(X, y):
    
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        
        
        model.fit(X_train, y_train)
        scores.append(accuracy_score(y_test, model.predict(X_test)))
    
    return np.mean(scores)

k_vals = [1, 3, 5, 7, 9]
p_vals = [1,2]
scores = []

for k in k_vals:
    score = []

    for p in p_vals:
        model = KNeighborsClassifier(n_neighbors = k, p = p)
        score.append(cv(model, X1, y1))
    
    scores.append(score)

print("Cross-validation scores:")
scores

Cross-validation scores:


[[0.9233333333333335, 0.8911111111111112],
 [0.9344444444444445, 0.8577777777777778],
 [0.8933333333333333, 0.8799999999999999],
 [0.8711111111111111, 0.8699999999999999],
 [0.8488888888888889, 0.8711111111111111]]

Here above is the cross validation scores for k and p values. The best score is 0.9344444444444445 which corresponds to k = 3 and p = 1. Knn classifier did better than the penalized logistic regression in terms of cv score.

### Second Representation

This representation also takes advantage of the fact that we can surely know the labels of the instances whose bags are labeled as 0. However, instead of doing in-bag distance calculations, this time each bag is represented with the instance which is the furthest to the known zeros of the whole set. By known zeros, we mean all instances belonging to the bags with label 0. By doing this, we represent each bag with the instance which is the least similar one to the zeros and therefore check the existance of a class 1 instance in each bag.

In [29]:
def representation2(bag):
    bag_array = np.array(bag)
    class_id = bag_array[0,0]
    bag_id = bag_array[0,1]
    
    data_zeros = data[data["Class"] == 0].drop(columns = ["Class", "Bag"])
    data_zeros = np.array(data_zeros)
    
    features = np.array(bag.drop(columns = ["Class", "Bag"]))
    
    dist_to_zeros = euclidean_distances(features,data_zeros,squared = False)
    
    most_distant_point = np.argmax(np.sum(dist_to_zeros, axis = 1))
    
    return np.concatenate((np.array([class_id,bag_id]), features[most_distant_point,:]))

In [30]:
rep2 = np.zeros((92,168))
for bag_id in range(1,93):
    bag = data[data["Bag"]==bag_id]
    rep2[bag_id - 1,:] = representation2(bag)

In [31]:
X2 = rep2[:,2:]
y2 = rep2[:,0]
model2 = LogisticRegressionCV(cv=10, random_state=0,
                              max_iter = 1000).fit(X2, y2)


In [32]:
print("Penalization terms:")
print(model2.Cs_)
print()
print("Corresponding average cross-validation scores:")
print(np.average(model2.scores_[1].T,axis = 1))
print()
print("Best penalization term")
print(model2.C_)

Penalization terms:
[1.00000000e-04 7.74263683e-04 5.99484250e-03 4.64158883e-02
 3.59381366e-01 2.78255940e+00 2.15443469e+01 1.66810054e+02
 1.29154967e+03 1.00000000e+04]

Corresponding average cross-validation scores:
[0.79777778 0.80777778 0.80777778 0.78555556 0.78555556 0.78555556
 0.76444444 0.76444444 0.76444444 0.76444444]

Best penalization term
[0.00077426]


Here above is the cross-validation scores for penalized logistic regression and most suitable penalty term. 0.80777778 is the corresponding cv score. Notice that the model performance has decreased when we changed to the second representation. This might suggest that for penalized logistic regression the first representation might be a better choice.

#### KNN Classifier Applied to the Second Representation

Same parameters will be tried.

In [36]:
k_vals = [1, 3, 5, 7, 9]
p_vals = [1,2]
scores = []

for k in k_vals:
    score = []

    for p in p_vals:
        model = KNeighborsClassifier(n_neighbors = k, p = p)
        score.append(cv(model, X2, y2))
    
    scores.append(score)
    
print("Cross-validation scores:")
scores

Cross-validation scores:


[[0.7922222222222223, 0.7577777777777779],
 [0.8033333333333333, 0.7822222222222222],
 [0.7288888888888889, 0.7277777777777779],
 [0.74, 0.7066666666666667],
 [0.7277777777777779, 0.6744444444444444]]

Here above is the cross validation scores for k and p values. The best score is 0.8033333333333333 which corresponds to k = 3 and p = 1. Overall, this model seems to be the worst one above all alternatives.

### Conclusion

All in all, we tried two different representations to solve the multiple instance learning problem and tried two different classifiers to evaluate them. Each representation was suggested considering that the instances from class 0 bags can be used to evaluate other ones. One built was upon in-bag comparisons and other was built upon global class 0 comparisons. 

By looking at the cv scores, we can say that the first one seems to be a better representation. However, it has a limitation which needs to be considered. This representation fails to represent the bag corretly if class 1 instances are at majority since it will choose a class 0 instance as the representative of the class. 